<a href="https://colab.research.google.com/github/fdsig/A-Lamp/blob/master/AVA_Image_Binary_Classification_aumentaions_mod_debuged.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
import shutil
import pandas as pd
import numpy as np
import zipfile
import random
from shutil import copyfile
import albumentations as A
import cv2
import matplotlib.pyplot as plt
from tqdm import tqdm
import io
import time
import os
import subprocess
import sys
from tqdm import tqdm
from torchvision import datasets, transforms
import torchvision
from PIL import Image
from torchvision.utils import save_image




In [2]:

def run_process(process = None, command = None):
    print(command)
    logname = process+'.log'
    env = os.environ.copy()
    with io.open(logname, 'wb') as writer, io.open(logname, 'rb', 1) as reader:
        process = subprocess.Popen(command, stdout=writer, shell=True, env=env)
        while process.poll() is None:
            sys.stdout.write(reader.read())
        # Read the remaining
        sys.stdout.write(reader.read())

## to innitialize 
depend = [ 
                'torch==1.7.0', 'torchvision==0.8.1', 'timm==0.3.2', 
                'timm==0.3.2', 'albumentations==0.4.6'
                ]
for dep in depend:
    run_process(command='pip install '+dep, process = dep.split('==')[0])

cleanup = False
if cleanup:
    current = [fid for fid in os.scandir() if os.path.isfile(fid)]
    for i in current:
        os.system('rm -rf '+i.name)


#!git clone https://github.com/fdsig/convit.git

#!git clone https://github.com/facebookresearch/convit.git
repositories = ['https://github.com/fdsig/convit.git',
                ' https://github.com/fdsig/image_utils',
                'https://github.com/fdsig/A-Lamp']
for repo in repositories:
    run_process(command='git clone '+repo, process=repo.split('/')[-1])

convit_fids = [fid for fid in os.scandir('convit/')]
other_fids = [fid for fid in os.scandir('image_utils/')]
for fid in convit_fids+other_fids:
    shutil.move(fid.path,fid.name)
os.system(
    'rm convit && rm image_utils && rm -rf convit &&\
     rm -rf MPADA && rm -rf image_utils')

import image_getter
pull = image_getter.Get_Ava()
pull.parse_urls()
pull.google_getter()
pull.ava_txt()
pull.download_ava_files(own_drive=True, 
                        download=True, 
                        full=True, 
                        clear_current=False)


pip install torch==1.7.0
pip install torchvision==0.8.1
pip install timm==0.3.2
pip install timm==0.3.2
pip install albumentations==0.4.6
git clone https://github.com/fdsig/convit.git
git clone  https://github.com/fdsig/image_utils
git clone https://github.com/fdsig/A-Lamp
https://drive.google.com/file/d/1YtU0m8cf2qgYcSpcPqlSz2GxO7wowu6W/view?usp=sharing


 The files are : ./batch_meta/ava_files_urls.txt
File IDs have sucessfully been obtaniend and now in content/batch_meta/
['1-9PbytN2awBviMP4Br8b4puhzFbUmOg2', '1-TcM8ZitdD-8yxnOlWDYcAIjszxDF5nH', '10hfdjtZgStNXMOYynLoIxlhiaItlAUQ4', '168mr0mkR6_5S5FoBkbzvUw8ulf-17D34', '16JJKoBivo0NUCcA_y2ueX2E0qFzWost7', '18ZMIqTlpfe7bm2cc6mgSkzfsxsmkeSTn', '1EimTfOAOnAsWLyzOESWVstKUcMJ6Ofqa', '1FRkogxsjjzaKumr-xYIp9rL_cLT01TR-', '1JkMJUIAyG3rn5eJ6VX8X5Z4Qq47UFqOi', '1KICR4a-a74aX6mSv9aAD3iXHV2n8vOtg', '1KQfOsOE1I5piQvEaVigYjw6do9Wn3Voz', '1M681Etlo_pot-p61f-wr8Uqf1J62XmuU', '1MifQhi1UJA255ulGYXz-_mWshDWdxrlv', '1NCJNl0gDHBC_cr-G026jIVrtrHgHcb62', '

100%|██████████| 255508/255508 [00:10<00:00, 24188.17it/s]


0 already exist 
 all files = 255508


45it [00:00, 5105.60it/s]


255508 [<DirEntry 'images'>]


In [3]:

class Square:
    def __call__(self,img): 
            dims = np.array(img.size)
            m_x = dims[[dims.argmax()]][0]
            y_axis_pad = int(m_x-dims[1])//2 ;x_axis_pad = int(m_x-dims[0])//2
            y_axis_pad+=1;x_axis_pad+=1
            pad = (x_axis_pad, y_axis_pad, x_axis_pad, y_axis_pad)
            return torchvision.transforms.functional.pad(img,pad,0,'constant')

train_reflect_tansform = A.Compose([
            A.augmentations.transforms.LongestMaxSize(max_size=256),
            A.augmentations.transforms.PadIfNeeded(256,256,  always_apply=True),
            ]) 
train_transforms = transforms.Compose(
    [
        Square(),
     transforms.Resize((256,256)),
    ]
)


In [4]:
def meta_process():
    df = pd.read_csv('ava_meta_with_int_id_230721.csv')
    y_gt = df['mos_float'].values
    ids = df['ID'].values
    all_ims = len(list(os.scandir('Images/images')))
    print()
    print(f' Initial Fids {len(ids)}')
    y_gt_std, y_gt_mean = np.std(y_gt, axis = 0), np.mean(y_gt, axis = 0)
    exclude_below = y_gt_mean-y_gt_std*4
    exclude_above = y_gt_mean+y_gt_std*4
    ids = ids[np.argwhere(y_gt>=exclude_below)].ravel()
    y_gt = y_gt[np.argwhere(y_gt>=exclude_below)].ravel()
    print(f' number image after outliers {len(y_gt)}')
    ids = ids[np.argwhere(y_gt<=exclude_above)].ravel()
    y_gt = y_gt[np.argwhere(y_gt<=exclude_above)].ravel()
    print(len(ids),len(y_gt))
    ids_low = ids[np.argwhere(y_gt<4.99)].ravel().astype(int)
    ids_high = ids[np.argwhere(y_gt>5.01)].ravel().astype(int)
    to_include = np.concatenate((ids_low,ids_high), axis=0)
    n_include = len(to_include)
    f' number of images to include {n_include} excluded {all_ims-n_include}'
    return df[df['ID'].isin(to_include)]

In [5]:
class Pre_pipe:
    def __init__(self):
        
        self.subpatch=False, 
        self.move=False
        self.box_n = False
        self.src = 'Images/images/'
        self.dest = 'data/'
        self.miss = None
        #create desitination paths
        self.test_train = ['train/','test/', 'val/']
        self.classes = ['class_1/','class_0/']
        self.create_dict()
    
    def destination(self):
        for i in self.test_train:
            for j in self.classes:
                os.makedirs(self.dest+i+j, exist_ok=True)
                print(f'made dir = {self.dest+i+j}')
        
    def create_dict(self):
        self.destination()
        train_dir,test_dir,val_dir = self.test_train
        class_1, class_0 = self.classes 
        src_did = self.src
        df = meta_process()
        y_g = df[['threshold','ID','set']].to_dict('index')
        y_g_dict = {
            str(y_g[pair_key]['ID']):y_g[pair_key]for pair_key in y_g
            }
        y_g_dict = {
            key:
            {**y_g_dict[key],**{'src':src_did+key+'.jpg'}} 
             for key in y_g_dict
             } 
        self.y_g_dict = {
            key:{**y_g_dict[key], **{'dest':class_1}} 
                 if y_g_dict[key]['threshold']==1 
                 else {**y_g_dict[key], **{'dest':class_0}} 
                 for key in y_g_dict
                 }
        dict_n = len(self.y_g_dict)                 
        print(f' Images Dictionary has meta data on {len(self.y_g_dict)} images')
        

    def copy(self, subpatch=None, move=None):
        
        if not self.move and self.subpatch:
            print('exected')
            box = f'_patch_{self.box_n}{ext}'    
            if self.miss:
                miss_copy = (k 
                    for i in os.scandir('data/') 
                    for j in os.scandir(i) 
                    for k in os.scandir(j))
                for fid in tqdm(miss_copy):
                    shutil.move(fid.path,self.root+fid.name)
                
            ##some process
            for key in tqdm(y_g_dict):
                
                if y_g_dict[key]['set'] == 'training':
                    self.y_g_dict[key]['fid']=(
                        root+train_dir+y_g_dict[key]['dest']+key+box
                        )
                elif y_g_dict[key]['set'] == 'test':
                    self.y_g_dict[key]['fid']=(
                        root+test_dir+y_g_dict[key]['dest']+key+box
                        )
                else:
                    self.y_g_dict[key]['fid']=(
                        root+val_dir+y_g_dict[key]['dest']+key+box
                        )
                
                if not os.path.isfile(self.y_g_dict[key]['fid']): 
                    print('exected')
                    src = y_g_dict[key]['src']
                    bbox = self.y_g_dict[key]['BBoxes'][self.box_n]
                    self.img = read(src)[bbox[1]:bbox[3], bbox[0]:bbox[2],:]

                    try:
                        cv2.imwrite(self.y_g_dict[key]['fid'],self.img)
                    except:
                        print(self.y_g_dict[key]['fid'], 'not saved')
    
    def augment(self,  augment_type=None):
        read = lambda fid_: cv2.cvtColor(cv2.imread(fid_), cv2.COLOR_BGR2RGB)
        y_g_dict = self.y_g_dict
        root = self.dest
        ext = '.jpg'
        train_dir,test_dir,val_dir = self.test_train
        for key in tqdm(y_g_dict):
            
            if augment_type == 'zero_pad':
                img = read(y_g_dict[key]['src'])
                img = train_transforms(Image.fromarray(img))
                img = np.asarray(img).astype('uint8')
            elif augment_type == 'reflect':
                img = read(y_g_dict[key]['src'])
                img = train_reflect_tansform(image=img)['image']

            if y_g_dict[key]['set'] == 'training':
                y_g_dict[key]['fid']=(
                    root+train_dir+y_g_dict[key]['dest']+augment_type+key+ext
                    )
                cv2.imwrite(y_g_dict[key]['fid'],img)
            elif y_g_dict[key]['set'] == 'test':
                y_g_dict[key]['fid']=(
                    root+test_dir+y_g_dict[key]['dest']+augment_type+key+ext
                    )
                cv2.imwrite(y_g_dict[key]['fid'],img)
            else:
                y_g_dict[key]['fid']=(
                    root+val_dir+y_g_dict[key]['dest']+augment_type+key+ext
                    )
                cv2.imwrite(y_g_dict[key]['fid'],img)
        
    def undo(self):
        fids = [k for i in os.scandir('data/') 
                  for j in os.scandir(i) 
                  for k in os.scandir(j)]
        
        dest = 'Images/images/'
        all_ims = len(list(os.scandir(dest)))
        print(f'there are {all_ims} ins Source')
        for i in fids:
            shutil.move(i.path,dest+i.name )
        fids = [k for i in os.scandir('data/') 
                  for j in os.scandir(i) 
                  for k in os.scandir(j)]
        all_ims = len(list(os.scandir(dest)))
        print(f'there are {all_ims} ins Source')
        

                
                
    def get_patches(self):
        self.subpach = True
        bbox_path = '/content/A-Lamp/multi_patch_subnet/adaptive_patch_selection/output-split'
        batches_dir = list(os.walk(bbox_path))[0][1]
        bbox_files = {i[-2:]:[j.path for j in os.scandir(bbox_path+'/'+i)] for i in batches_dir}
        keys = list(bbox_files.keys())
        for key in tqdm(keys):
            for fl in bbox_files[key]:
                with open(fl, 'rb') as pk:
                    if 'df' not in locals():
                        df = pd.read_pickle(pk)
                    else:
                        df = df.append(pd.read_pickle(pk))
                    
        boxes_dict = {
            k.split('/')[-1].split('.')[0]:{'BBoxes':v} 
            for k,v in zip(df['Filename'],df['BBoxes']) 
            }

        self.y_g_dict = {
            key:{**self.y_g_dict[key],**boxes_dict[key]} 
            for key in self.y_g_dict 
            if key in boxes_dict
            }




In [6]:
#pre = Pre_pipe()
pre_args = ['reflect','zero_pad']
import copy 
import shutil
# consider using from pathlib import Path to adress multipl +'/' in code
class Implement:
    def __init__(self):
        #self.pre = Pre_pipe()
        self.convit_args = [
                'python main.py',
                '--pretrained', 
                '--repeated-aug',
                '--batch-size 260' ,
                '--data-path data/',
                '--nb_classes 2', 
                '--nb_classes_pre 1000', 
                '--input-size 224',  
                '--embed_dim 48', 
                '--world_size 1', 
                '--epochs 10',
                '--warmup-epochs 5',
                '--cooldown-epochs 0',
                '--lr-noise-pct 0.5',
                '--lr-noise-std 0.5',
                '--decay-epochs 0',
                '--pre_save',
                '--pre_save_path drive/MyDrive/0.AVA/results/convt_tiny_zero_stt/', 
                '--resume drive/MyDrive/0.AVA/results/convt_tiny_zero_stt/checkpoint.pth'

                ]
        self.fid = 'drive/MyDrive/0.AVA/results/grids'
    def augment(self):
        Pre_pipe().augment(augment_type='reflect')

    def grid(self):
        args = {"--color-jitter" : np.round(np.linspace(.2,.5,3),2),
        '--locality_strength': np.round(np.linspace(0.5,2,6),2),
        '--lr': np.round(np.linspace(0.0001,0.0003,3),5)}
        cat, local, lr = (args[i] for i in args)
        search_space = [[
        cat[i],local[j],lr[m]] 
        for i in range(len(cat)) 
        for j in range(len(local))
        for m in range(len(lr))]

        print(f'Searching {len(search_space)} Unique Paramiter Combinations ')
        
        return [' '.join([key+' '+str(arg) for key, arg in zip(args,i)]) for i in search_space]
    
    
    def clear_cach(self):
        chk_dir = '/root/.cache/torch/hub/checkpoints/'
        if os.path.isdir(chk_dir):
            for chkpnt in os.scandir(chk_dir):
                print(f'rm"ing {chkpnt.path}')
                os.system(f'rm {chkpnt.path}')


    def run(self,compare_size=None,grid_search=None, convit_args=None, ):
        pre_args = ['reflect','zero_pad']
        model_args = [ '--model convit_tiny', '--model convit_small', '--model convit_base']
        if compare_size:
            for arg in pre_args:
                os.system('rm -rf data/')
                pre.destination()
                pre.move == True
                pre.miss == False
                print(f'Augmenting with: {arg}')
                pre.augment(augment_type=arg)
                for size in model_args:
                    self.convit_args.insert(4, '--output_dir drive/MyDrive/0.AVA/results/'+arg+size.split(' ')[-1])
                    self.convit_args.insert(3,size)
                    command = ' '.join(self.convit_args)
                    print(f'Runing Process with Args : {self.convit_args}')
                    run_process(process='convit', command=command)
                    convit_args.pop(3);convit_args.pop(4)
        

        elif grid_search:
            
            grid = self.grid()
            self.convit_args.insert(3,'--model convit_tiny')
            convit_args = self.convit_args
            args_check = copy.deepcopy(convit_args)
            print(convit_args)
            
            for idx, grid_i in enumerate(grid):
                #append hyper params args
                convit_args.append(grid_i)
                #checks for cached model weights
                # to mitigate data leak if subprocess code 
                # reloading model on weights grid n -=1(previous grid). 
                self.clear_cach()

                print('\n',idx,grid_i)
                #subprocess directory 
                dir_arg = '--output_dir '
                
                grids_dir = 'drive/MyDrive/0.AVA/results/grids_2/'
                self.dir =grids_dir
                grid_n = 'grid_'+ str(idx)

                model_dir = grids_dir + grid_n + '/'
                convit_args.insert(4,dir_arg+ model_dir)
                print(f'model args including model dir is{"%"*20} {convit_args}')

                resume_from_fid = 'drive/MyDrive/0.AVA/results/convt_tiny_zero_stt/'

                # file in directory of grid_i path names
                checkpoint,log = 'checkpoint.pth','log.txt'
                resume_from = resume_from_fid+checkpoint
               
                # for restarting from logfile
                log_fid = resume_from_fid+log

                # check if chepoint file exists
                if os.path.isfile(resume_from):
                    self.clear_cach()
                    cmd = '--resume '
                    print(f'resuming from {resume_from}')
                    convit_args.insert(4,cmd+resume_from)
                    # check if log file to get n epochs
                    if os.path.isfile(log_fid):
                        with open(log_fid, 'r') as fid:
                            n_epochs = len(fid.readlines())
                    else:
                        n_epochs = 0

                    if n_epochs<=10:
                        n_epochs-=1
                        print('n_epochs',n_epochs)
                        start = "--start_epoch "+str(n_epochs)
                        convit_args.insert(4,start)
                        #joins suprocess args list to string
                        # for subprocess args parser. 
                        command = ' '.join(self.convit_args)
                        print(f'Running: {command}')
                        # runs subprosess 
                        run_process(process='covit_gird_'+str(idx), 
                                    command=command)
                        self.clear_cach()
                        
                        convit_args.pop(4);convit_args.pop(4)
                    
                    elif n_epochs>=10:
                        print('passing as already trained', '*'*20)
                        convit_args.pop(4)
                    else:
                        # runs code as normal withot resum 
                        print('running as normal'* 30, '\n')
                        print(convit_args)
                        ## insert resume from path of base(zero train)
                        
                        convit_args.pop(4)
                        print(convit_args)
        
                        command = ' '.join(self.convit_args)
                        print(f'running: {command}')
                        run_process(process='covit_gird_'+str(idx), 
                                    command=command)
                        self.clear_cach()


                else:
                    command = ' '.join(self.convit_args)
                    print(f'running: {command}')
                    run_process(process='covit_gird_'+str(idx), command=command)
                    self.clear_cach()
                convit_args.pop(4)


                #rmoves grid_i params
                convit_args.pop()
                # checks if params have been reset(for debugging)
                print(convit_args == args_check)
    def get_metrics(self):
        ''' for evaluating paramiter search (copies log.txt ans saves idx'd version'''
        #resolve grid = file name (where in code???) 
        logs = [
         (j,_idx) for _idx,i in enumerate(os.scandir(self.dir)) 
            for idx,j in enumerate(os.scandir(i)) if 'log.txt' in j.name
            ]
        for posix,idx in logs:
            path = '/'.join(posix.path.split('/')[:-2])+'/grid_all_logs/'
            os.makedirs(path,exist_ok=True)
            name,ext = posix.name.split('.')
            new_name = name+str(idx)+'.'+ext
            new_fid = path+new_name
            #print(new_fid)
            shutil.copy(posix,new_fid)
        
    

            


In [ ]:
Implement().augment()
Implement().run(grid_search=True)

made dir = data/train/class_1/
made dir = data/train/class_0/
made dir = data/test/class_1/
made dir = data/test/class_0/
made dir = data/val/class_1/
made dir = data/val/class_0/

 Initial Fids 255502
 number image after outliers 255411
255403 255403
 Images Dictionary has meta data on 253030 images


  3%|▎         | 8546/253030 [01:23<34:12, 119.13it/s]

In [ ]:
 chk_dir = '/root/.cache/torch/hub/checkpoints/'

if os.path.isdir(chk_dir):
    for chkpnt in os.scandir(chk_dir):
        print(f'rm"ing {chkpnt.path}')
        os.system(f'rm {chkpnt.path}')

!python main.py --pre_save --pre_save_path drive/MyDrive/0.AVA/results/convt_tiny_zero_stt/  --pretrained --batch-size 260 --model convit_tiny --epochs 20 --output_dir drive/MyDrive/0.AVA/results/convt_tiny_zero_stt/ --data-path data/ --nb_classes 2 --nb_classes_pre 1000 --input-size 224

In [ ]:
aa='rand-m9-mstd0.5-inc1', 
batch_size=50, 
clip_grad=None, 
color_jitter=0.4, 
cooldown_epochs=10, 
cutmix=1.0, 
cutmix_minmax=None, 
data_path='data/', 
data_set='IMNET', 
decay_epochs=30, 
decay_rate=0.1, 
device='cuda', 
dist_url='env://', 
distributed=False, 
drop=0.0, 
drop_block=None, 
drop_path=0.1, 
embed_dim=48, 
epochs=20, 
eval=False, 
inat_category='supercategory', 
input_size=224, 
local_up_to_layer=10, 
locality_strength=4.0, 
lr=0.0005, 
lr_noise=None, 
lr_noise_pct=0.67, 
lr_noise_std=1.0, 
min_lr=1e-05, 
mixup=0.8, 
mixup_mode='batch', 
mixup_prob=1.0, 
mixup_switch_prob=0.5, 
model='convit_base', 
model_ema=False, 
model_ema_decay=0.99996, 
model_ema_force_cpu=False, 
momentum=0.9, 
nb_classes=2, 
nb_classes_pre=1000, 
num_workers=10, 
opt='adamw', 
opt_betas=None, 
opt_eps=1e-08, 
output_dir='drive/MyDrive/0.AVA/results/zero_padconvit_base', 
patience_epochs=10, 
pin_mem=True, 
pretrained=True, 
recount=1, 
remode='pixel', 
repeated_aug=True, 

resplit=False, 
resume='', 
sampling_ratio=1.0, 
save_every=None, 
sched='cosine', 
seed=0, 
smoothing=0.1, 
start_epoch=0, 
train_interpolation='bicubic', 
warmup_epochs=5, 
warmup_lr=1e-06, 
weight_decay=0.05, 
world_size=1

In [ ]:
'''boxes = [0,1,2,3,4]
for box in boxes:
    os.system('rm -rf data/')
    print('data' in [i.name for i in os.scandir('.')])
    pre.box_n=box
    pre.destination()
    print('data' in [i.name for i in os.scandir('.')])
    pre.miss = True
    pre.copy()
    args[4]='--output_dir drive/MyDrive/0.AVA/results/patch_no_'+str(box)
    command = ' '.join(args)
    fid = list(os.scandir('data/test/class_0/'))[1].path
    img = cv2.imread(fid)
    plt.imshow(img)
    run_process(process = 'convit', command = command)'''

In [ ]:
 fid_ = pre.y_g_dict[list(pre.y_g_dict.keys())[5]]['src']
 read = lambda fid_: cv2.cvtColor(cv2.imread(fid_), cv2.COLOR_BGR2RGB)
 img = read(fid_)
 zero_pad_array = train_transforms(Image.fromarray(img)).permute(1,2,0)
 plt.imshow(zero_pad_array)